In [ ]:
#api="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"
from langchain_groq import ChatGroq
llm = ChatGroq(groq_api_key=api, model_name="llama-3.2-3b-preview")

In [2]:
llm.invoke("what is force")

AIMessage(content="Force is a fundamental concept in physics that describes a push or pull that causes an object to change its state of motion or shape. It is a measure of the interaction between two objects, and it depends on the mass of the objects, the distance between them, and the acceleration they produce.\n\nThere are several types of forces, including:\n\n1. **Gravitational force**: The force of attraction between two objects with mass, such as the Earth and an object on its surface.\n2. **Frictional force**: The force that opposes motion between two surfaces that are in contact, such as the force that prevents a car from slipping on a wet road.\n3. **Normal force**: The force exerted by a surface on an object in contact with it, such as the force that presses a book onto a table.\n4. **Tensile force**: The force that stretches or pulls apart two objects, such as the force that holds a rope or a piece of string.\n5. **Compressive force**: The force that compresses or squeezes t

In [3]:
from sentence_transformers import SentenceTransformer, util
from datetime import datetime
import re
import chromadb
import ollama

from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
oembed = OllamaEmbeddings(base_url="http://localhost:11434", model="nomic-embed-text")
collection_name="phydb"  ##phydbllama3


persist_directory="database"
vectorstore = Chroma(
     collection_name=collection_name,
    embedding_function=oembed,
    persist_directory=persist_directory
)
retriever = vectorstore.as_retriever()

c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\importlib\__init__.py:126: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  return _bootstrap._gcd_import(name[level:], p

In [2]:
def llmRagQuery(prompt):
    model = ollama.generate(
        model="llama3.2",
        system="""
        you are a rag assistant who will rewrite the question/prompt given by the user.
        you will rewrite 3 more times.
        the rewritten prompts/questions has to be exactly the same topic as that of the user's.
        it can't be too advanced nor too simplistic compared to the user's
        you will not include anything else in your answer.
        list the rewritten questions/prompts in an ordered list

        """,
        prompt=prompt,
    )

    newPrompts = prompt + " "
    llmPrompts = re.compile(r"([0-9 .]+)(.+)")
    llmPrompts = re.finditer(llmPrompts,model["response"])

    for prompts in llmPrompts:
        newPrompts += prompts.group(2) + " "
    result = retriever.invoke(newPrompts)
    str = " "
    for i in result:
        str+= i.page_content
    return str
modelMini = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def check_Meaning(old_response, new_response):
    # embed the texts
    embeddings = modelMini.encode([old_response, new_response])


    similarity = util.cos_sim(embeddings[0], embeddings[1])
    
    if similarity.item()*100 > 50:
        return new_response

    else:
        return old_response

c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
import json
path ='dataset/fact/allfact.json'
with open(path) as f:
  json_data = json.load(f)

In [8]:
def SaveDataset(data, path=None, fileName="syntheticDatafactall"):
  if path is None:
      filePath = f"{fileName}.json"
  else:
      filePath = f"{path}/{fileName}.json"
  with open(filePath, 'a') as f:
      json.dump(data, f, indent=4)
  print("Saved Successfully!")

In [6]:
from tqdm import tqdm
p=[]
for i in tqdm(json_data):
    message = i['question']
    query = llmRagQuery(message)
    tmp = check_Meaning(message,query)
    context = " "
    if tmp == message:
        tmp = "None "

    else:
            context = tmp
    system ="""You are an A.I. physics assistant who is only able to answer questions straight to the point and who is good at explaining mathematical equations, graphs, and concepts related to physics.
                you are going to answer in a way that is easier for 9th grade students to understand.
                If you do not know something, then say so; otherwise, answer concisely and accurately.
                Absolutely don’t repeat system context, or userPrompt in your response.
                if the user provides context, then use that context to answer the question but it has to be relevant to the user prompt otherwise say you dont know.
            """
    prompt = f"""
        "Context": {context},
        "UserPrompt": {message}
    """

    response = ollama.generate(
    model="llama3.2",
    system=system,
    prompt=prompt

    )
    j ={
    "question":message,
    "answer":response['response'],
    "referenceAnswer": i['referenceAnswer'],
    "referenceContext": i['referenceContext'],
    "context": context
    }
    p.append(j)
SaveDataset(p)

  0%|          | 0/864 [00:00<?, ?it/s]

c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
100%|██████████| 864/864 [4:16:50<00:00, 17.84s/it]  


Saved Successfully!


In [ ]:

import requests

# Your OpenRouter API key
api_key = "sk-or-v1-af7953477f9e13e5d40bd26b631f4a6525a0e05d8ffd0bd30f420e077275f08"

# OpenRouter endpoint and model
endpoint = "https://openrouter.ai/nousresearch/hermes-3-llama-3.1-405b:free/api"
# Define the request payload
payload = {
    "prompt": "Tell me a joke",
    "max_tokens": 100
}

# Set the headers, including the API key
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

# Make the POST request to OpenRouter
response = requests.post(endpoint, json=payload, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Print the response from the model
    result = response.json()
    print(result["text"])
else:
    print(f"Request failed with status code {response.status_code}: {response.text}")

In [11]:
import json
path ='conversation/chap4.json'
with open(path) as f:
  json_data = json.load(f)

In [13]:
p=[]
for i in json_data:
    a = LLMTestCase(
            input=i["input"],
            actual_output=i["actual_output"] 
                          )
    p.append(a)

In [14]:
p

[LLMTestCase(input='When does work happen if I hold a book still in my hand?', actual_output='Great question. Holding a book still, thereâ€™s no displacement, so by the scientific definition, no work is done. But if you moved the book from one place to another, that would count as work. Can you think of examples of work in everyday life?', expected_output=None, context=None, retrieval_context=None, additional_metadata=None, comments=None, tools_called=None, expected_tools=None, reasoning=None, name=None),
 LLMTestCase(input='Yes, but I thought work happened even if I just held something heavy for a long time.', actual_output="Thatâ€™s understandable since it feels like you're expending energy, but in physics, work requires displacement along with force. Imagine pushing a cart or lifting a weight; in those cases, thereâ€™s both force and movement, so we can measure work.", expected_output=None, context=None, retrieval_context=None, additional_metadata=None, comments=None, tools_called=N